In [17]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
import copy
import numpy as np

import pickle
from scipy.integrate import simps
from scipy.interpolate import RectBivariateSpline
from classy import Class
import sys
sys.path.append("../scripts/") 
from postBornCrossSpectra import PostBorn_Bispec
import Cosmology as C
import kernels
plt.style.use('classic')

In [18]:
from lab import *

In [19]:
plotpath = '../output/plots/'

In [20]:
zmin = 1e-5
zmax = 1090.

kmin = 1e-4
kmax = 50

#cosmo_dict.pop('z_cmb')
cosmo = [{'name':"EmasCosmo"},cosmo_dict]

print(cosmo_dict)

a     = np.linspace(1./(1.+zmin),1./(1.+zmax),500)  
z     = 1./a-1.

data  = C.CosmoData(cosmo[1],z)

# L     = np.logspace(1,4,400)

# # sampling for integrations
# phi = np.linspace(0.,2.*np.pi,1000,endpoint=True)
# L1  = np.logspace(-1,5,400)

# use same sampling as in Limber case


LSST_bins  = ['all']#0,1,2,3,4,

{'A_s': 2.10732e-09, 'h': 0.677, 'k_pivot': 0.05, 'n_s': 0.96824, 'omega_b': 0.02247, 'omega_cdm': 0.11923}
computing distances, derived parameters...
{'Neff': 3.046}
{'h': 0.677}
{'m_ncdm_tot': 0.0}
{'omega_m': 0.6745515980724002}


In [21]:
# define redshift bins (LSST-like)
zmins= [0,0.5,1,2,3,0]
zmaxs= [0.5,1.,2.,3.,4.,'z$_{\mathrm{CMB}}$']
ngal = [14.3,23.9,21.6,3.7,1.5,65.7]

colors=['navy','maroon', 'darkgreen', 'mediumaquamarine','indianred','darkorange']

In [22]:
#compute correction following Krause&Hirata
#always use trapz rule for accurate results!
#redefinition with l'-> l'-l
def get_regularized_sum_of_terms(L,L1,phi,M,method='trapz'):
    nu = np.cos(phi)
    res=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            L_= LL*np.sqrt(1.+(LL1/LL)**2+2.*LL1/LL*nu)
            integrand =LL*LL1**3*(LL1*nu+LL)*(LL1+LL*nu)**2*(M(L_,LL1,grid=False)/L_**2/LL1**4-M(LL,LL1,grid=False)/LL**2/LL1**4)
            if method =='simps':
                resL1+=[simps(integrand,phi)]
            elif method =='trapz':
                resL1+=[np.trapz(integrand,phi)]
            elif method =='sum':
                resL1+=[np.sum(integrand*np.diff(phi)[0])]
        res+=[simps(resL1,L1)]
    res=4.*np.asarray(res)/(2*np.pi)**2
    return res


#numerical angular integration
def get_second_sum_of_terms(L,L1,M,method='trapz'):
    nu = np.cos(phi)
    res=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            integrand = LL1*(LL*LL1**2*(-LL1*nu+LL)*(-LL1+LL*nu)**2-LL**2*(LL*LL1*nu)**2)*(M(LL,LL1,grid=False)/LL**2/LL1**4)
            if method =='simps':
                resL1+=[simps(integrand,phi)]
            elif method =='trapz':
                resL1+=[np.trapz(integrand,phi)]
            elif method =='sum':
                resL1+=[np.sum(integrand*np.diff(phi)[0])]
        res+=[simps(resL1,L1)]
    res=4.*np.asarray(res)/(2*np.pi)**2
    return res

In [23]:
first_kernel  = kernels.CMB_lens(data.chi_cmb,data)
simple_kernel = kernels.CMB_lens(None,data)

Mstarspls = []
PBs =[]
Cls= []
for LSST_bin in LSST_bins:
    second_kernel = kernels.gal_clus(kernels.dNdz_LSST,kernels.simple_bias,data,LSST_bin)
    PB  = PostBorn_Bispec(data, zmin, data.z_cmb, first_kernel, second_kernel, simple_kernel, k_min=kmin,k_max=100, lmax=30000, acc=2)
    PBs+=[PB]
    Mstarspls+=[PB.Mstarsp]
    Cls+=[PB.CL_born]
ls = PB.ls


using z-bin None norm 65.75020607931236
Initializing CLASS with halofit...
{'A_s': 2.10732e-09, 'h': 0.677, 'k_pivot': 0.05, 'n_s': 0.96824, 'omega_b': 0.02247, 'omega_cdm': 0.11923, 'output': 'mPk', 'P_k_max_1/Mpc': 101, 'z_max_pk': 1088.6464513003662, 'k_min_tau0': 0.002, 'k_max_tau0_over_l_max': 5.0, 'k_step_sub': 0.015, 'k_step_super': 0.0001, 'k_step_super_reduction': 0.1, 'k_per_decade_for_pk': 20, 'perturb_sampling_stepsize': 0.01, 'tol_perturb_integration': 1e-06, 'halofit_k_per_decade': 3000.0, 'non linear': 'halofit'}
sigma8  0.822427245205793
zmax 1088.6464513003662


In [24]:
second_kernel = kernels.gal_clus(kernels.dNdz_LSST,kernels.constant_bias,data,'all')
chis = np.linspace(0,chi_cmb,1000)
np.trapz(second_kernel(chis,data.zchi(chis)),chis)

using z-bin None norm 65.75020607931236


0.9999988012417791

In [38]:
L     = np.logspace(np.log10(min(ell_)),3,200)
L1    = np.logspace(np.log10(min(ell_)),np.log10(max(ell_)),200)

In [36]:
L     = np.logspace(np.log10(min(ell_)),3,200)
L1    = np.logspace(np.log10(min(ell_)),np.log10(max(ell_)),200)

# sampling for integrations
phi = np.linspace(0.,2.*np.pi,1000,endpoint=True)

corrs=[]
for ii in range(len(LSST_bins)):
    print(ii),
    corrs+=[get_regularized_sum_of_terms(L,L1,phi,Mstarspls[ii],method='trapz')+get_second_sum_of_terms(L,L1,Mstarspls[ii],method='trapz')]
corrs = np.asarray(corrs)
np.save('../output/LimberPostBornResultsLSST_cut',(L,corrs))

0


In [26]:
#L,corrs = np.load('../output/LimberPostBornResultsLSST.npy')

In [27]:
#31 term only
def get_31_term_only(L,L1,M):
    return -L**2/np.pi*simps(M(L,L1)/L1,L1)

In [28]:
Cl31s=np.zeros_like(corrs)
for ii in range(len(LSST_bins)):
    Cl31s[ii] = get_31_term_only(L,L1,Mstarspls[ii])
np.save('../output/LimberPostBornCl31_kappa-gal',(L,Cl31s))

In [29]:
# getting this thing for uch 
# use same sampling as in Limber case

from lab import *
M_ = Mstarspls[-1](ell_,ell_)
#M_new = np.zeros_like(M_)

# for ii in range(len(ell_)):
#     for jj in range(len(ell_)):
#         if (ell_[ii]<=1000 and ell_[jj]<=2000):
#             M_new[ii][jj]=M_[ii][jj]
M_new = RectBivariateSpline(ell_,ell_,M_)

In [30]:


# sampling for integrations
phi = np.linspace(0.,2.*np.pi,1000,endpoint=True)

CL31_cut = get_31_term_only(L,L1,M_new)
np.save('../output/LimberPostBornCl31_kappa-gal_all_cut',(L,L1,CL31_cut))

In [39]:
LL1,LL2  = np.meshgrid(L1,L1)
M_corr = M_new(L1,L1)/LL1**4/LL2**2*4.

np.save('../output/MLimberSpline_bin_all_cut',[np.meshgrid(L1,L1),M_corr])

In [33]:
L     = np.logspace(1,4,400)

# sampling for integrations
phi = np.linspace(0.,2.*np.pi,1000,endpoint=True)
L1  = np.logspace(-1,5,400)
corrs=[]
for ii in range(len(LSST_bins)):
    print(ii),
    corrs+=[get_regularized_sum_of_terms(L,L1,phi,Mstarspls[ii],method='trapz')+get_second_sum_of_terms(L,L1,Mstarspls[ii],method='trapz')]
np.save('../output/LimberPostBornResultsLSST',(L,corrs))

0
